In [1]:
from cemba_data.atac.bigwig import frag_to_bw_batch
from cemba_data.atac.dump_fragment_bed import dump_frags
import pandas as pd
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed
import subprocess

## Make Cell Table - ATAC SubType

In [2]:
atac_cell_tidy_data = pd.read_msgpack('../Metadata/ATAC.cell_tidy_data.v2.msg')
atac_cell_tidy_data[['sample', 'barcode', 'SubType']].to_csv('CellTable.subtype.csv', index=None)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Dump fragments - ATAC SubType

In [3]:
sample_snap_dir = pathlib.Path('/gale/raidix/rdx-4/CEMBA_ATAC/snap/')
snap_dict = {i.name.split('.')[0]: str(i) for i in sample_snap_dir.glob('*.snap')}
pd.Series(snap_dict).reset_index().to_csv('SnapPath.csv', index=None, header=None)

In [ ]:
cpu = 40
cell_group_path = 'CellTable.subtype.csv'
output_dir_path = str(pathlib.Path().absolute())
sample_snap_path = 'SnapPath.csv'

# extract cluster fragments
frag_bed_path_list = dump_frags(cell_group_path,
                                output_dir_path,
                                sample_snap_path,
                                cpu=cpu)

## Merge fragments to mC SubTypes

In [4]:
import json
with open(
        '/home/hanliu/project/mouse_rostral_brain/metadata/label_transfer_to_atac_new_version/mC-ATAC.SubType.json'
) as f:
    sub_type_to_atac = json.load(f)

In [5]:
input_dir = pathlib.Path('SubType/').absolute()
output_dir = pathlib.Path('mCSubType/').absolute()
atac_input_dict = {'.'.join(p.name.split('.')[:-2]): str(p) for p in input_dir.glob('*bed.gz')}

In [ ]:
with ProcessPoolExecutor(10) as pool:    
    fs = []
    for mc_subtype, atac_subtype_list in sub_type_to_atac.items():
        cmd_l = ['zcat']
        for atac_cluster in atac_subtype_list:
            cmd_l.append(atac_input_dict[atac_cluster])
        cmd = ' '.join(cmd_l)
        cmd += f" | bgzip -c > {output_dir}/{mc_subtype.replace(' ', '_')}.bed.gz"
        f=pool.submit(subprocess.run, cmd, shell=True, check=True)
    
    for f in as_completed(fs):
        r = f.result()

## Bed to bigwig

In [6]:
frag_bed_path_list = [
    i for i in list(output_dir.glob('*bed.gz'))
    if not (output_dir / (i.name[:-6] + 'bw')).exists()
]
chrom_size_path = '/home/hanliu/ref/mouse/genome/fasta/raw/mm10.chrom.sizes'

In [ ]:
frag_to_bw_batch(frag_bed_path_list, chrom_size_path, remove_temp=True, cpu=20)

Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Sort fragment
Calculate COV
Generate bigwig
Sort fragment
